In [ ]:
pip install pyvi

In [ ]:
import unicodedata
import regex as re
from pyvi import ViTokenizer, ViPosTagger

bang_nguyen_am= [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']
nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

# Chuẩn hóa unicode 
# Có 2 loại unicode : unicode tổ hơp và unicode dựng sẵn, điêu này dẫn tới việc 2 từ giống nhau sẽ bị coi là khác nhau 
# Chuẩn hóa tất cả về 1 loại là unicode dựng sẵn
def chuan_hoa_unicode(text):
	text = unicodedata.normalize('NFC', text)
	return text

# Có 2 kiểu gõ dấu ở Tiếng Việt, ví dụ như là : òa hoặc oà (ta gọi lần lượt là chuẩn 1 và 2). Mặc dù kiểu gõ chữ sau ít 
#phổ biến hơn tuy nhiên vẫn cần phải chuẩn hóa tránh việc một số văn bản vẫn sử dụng kiểu gõ dấu thứ 2.
"""
	Hàm này xử lý chuẩn hóa từng từ một, sau khi chuẩn hóa từng từ thì ta sẽ đi chuân hóa từng câu sau 
	""" 
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# Tách từ tiếng việt, từ tiếng việt không giống như tiếng anh, tách từ tiếng anh ta chỉ cần tách bằng khoảng trắng
# Tuy nhiên từ tiếng Việt có cả từ đơn lẫn từ ghép nên tách từ tiêng Việt sẽ phúc tạp hơn 
# Project sử dung thu viện pyvi
def tach_tu_tieng_viet(text):
	text = ViTokenizer.tokenize(text)
	return text

# Đưa về chữ viết thường 
def chuyen_chu_thuong(text):
	return text.lower()

# Xóa đi các dấu cách thừa, các từ không cần thiết cho việc phân loại vẳn bản 
def chuan_hoa_cau(text):
	text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',text)
	text = re.sub(r'\s+', ' ', text).strip()
	return text
#html
def remove_html(text):
  return re.sub(r'<[^>]*>', '', text)

def tien_xu_li(text):
  text = chuan_hoa_unicode(text)
  text = chuan_hoa_dau_cau_tieng_viet(text)
  text = tach_tu_tieng_viet(text)
  text = chuyen_chu_thuong(text)
  text = chuan_hoa_cau(text)
  text = remove_html(text)
  return text

if __name__ == "__main__":
    file = open("/content/danhgia.txt", "r", encoding="utf-8")
    data = file.read()
    data = tien_xu_li(data)
    print("Ví dụ : \n")
    print(data)

Ví dụ : 

1 thiết_kế đẹp nhưng không hiểu sao lại để đám chữ 8 0 autofocus ở camera chi không biết nhìn hơi quê nếu không có thì sẽ đẹp hơn giờ mấy máy cao_cấp đâu ai để mấy thông_số đó lên máy đâu 1 công_nhận là không còn điện_thoại nào rẻ hơn điên thoại này v843 của viettel củng phải gọi là cụ 1 hiện_tại dùng prime 805 cày game mobile để qua đêm cắm sạc play game không bị văng 1 em nghĩ bác nên dùng qua mới biết được tuy pin 2k nhưng dùng được wifi 4h liên_tục về khoản pin này thì em chắc nhiều con của các hãng danh_tiếng chưa chắc đã bằng 1 pin khá hơn tí thì tốt nhỉ 1 gì cũng ngon trừ pin và brand pin 2500 thì ngon hơn 1 đã trên tay em này cầm không được sướng tay cho lắm nhưng được cái đẹp camera chấp_nhận được màu đen khá sâu máy này chỉ dành cho nam 1 em nó độ hoàn_thiện cũng khá good đấy 1 ngon nhỉ 1 theo bảng cấu_hình thì có_lẽ xử_lý đồ_họa rất tốt 1 đẹp đấy chứ nhìn nam_tính không có hình bản màu đen nhỉ 1 khá khen cho lenovo 1 phải công_nhận là đẹp nhưng không mua 1 công_nhậ

In [ ]:
# Thống kê số lượng data theo nhãn
count = {}
for line in open('/content/danhgia.txt'):
    key = line.split()[0]
    count[key] = count.get(key, 0) + 1

for key in count:
    print(key, count[key])

1 2627
0 1381


In [ ]:
# Thống kê các word xuất hiện ở tất cả các nhãn
total_label = 2
vocab = {}
label_vocab = {}
for line in open('/content/danhgia.txt'):
    words = line.split()
    # lưu ý từ đầu tiên là nhãn
    label = words[0]
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words[1:]:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

count = {}
for word in vocab:
    if len(vocab[word]) == total_label:
        count[word] = min([label_vocab[x][word] for x in label_vocab])
        
sorted_count = sorted(count, key=count.get, reverse=True)
for word in sorted_count[:100]:
    print(word, count[word])

. 805
không 562
, 510
thì 502
là 432
có 386
cái 346
mà 320
này 314
được 271
con 221
mình 219
của 207
mua 206
quá 202
nhưng 200
hình 198
cũng 197
máy 192
và 191
với 189
hơn 188
dùng 186
như 180
giá 177
nó 173
cho 172
em 170
gì 154
1 151
còn 143
pin 140
rồi 140
thấy 132
màn 126
) 124
lại 123
nhìn 118
chỉ 117
ra 116
nào 114
phải 114
mới 113
! 107
điện 102
thế 101
sao 96
nhiều 96
mấy 95
nên 94
làm 92
hay 91
2 90
nữa 90
cảm 89
lắm 88
chứ 88
thoại 87
bị 87
về 86
đi 86
đã 85
khi 83
biết 82
sẽ 82
ứng 82
giờ 81
hàng 79
: 78
thôi 78
vẫn 76
các 75
bác 73
thiết 73
để 72
cả 71
kế 69
lên 69
xài 69
nói 69
ko 69
bạn 68
đẹp 66
bằng 66
samsung 66
chắc 65
trên 65
dụng 65
hơi 64
chưa 64
rất 64
thích 63
thật 61
thể 61
đó 60
xem 59
vậy 59
người 59
từ 58
vào 57


In [ ]:
# loại stopword khỏi dữ liệu
# lưu file dùng về sau
stopword = set()
with open('stopwords.txt', 'w') as fp:
    for word in sorted_count[:100]:
        stopword.add(word)
        fp.write(word + '\n')
    
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)
    
    
with open('news_categories.prep', 'w') as fp:
    for line in open('/content/danhgia.txt'):
        line = remove_stopwords(line)
        fp.write(line + '\n')

In [ ]:
!head news_categories.prep

Thiết hiểu đám chữ " 8.0 autofocus " ở camera chi quê nếu Giờ cao cấp đâu ai thông số đâu
Công nhận rẻ điên V843 Viettel củng gọi cụ
Hiện tại Prime 805 cày game mobile qua đêm cắm sạc play game văng
Em nghĩ qua Tuy 2k wifi 4h liên tục khoản hãng danh tiếng
Pin khá tí tốt nhỉ
Gì ngon trừ brand Pin 2500 ngon
Đã tay cầm sướng tay camera chấp nhận Màu đen khá sâu dành nam
Em độ hoàn thiện khá good đấy
Ngon nhỉ "
theo bảng cấu lẽ xử lý đồ hoạ tốt


In [ ]:
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
test_percent = 0.2

text = []
label = []

for line in open('/content/danhgia.txt'):
    words = line.strip().split()
    label.append(words[0])
    text.append(' '.join(words[1:]))

X_train, X_test, y_train, y_test = train_test_split(text, label, test_size=test_percent, random_state=42)

# Lưu train/test data
# Giữ nguyên train/test để về sau so sánh các mô hình cho công bằng
with open('train.txt', 'w') as fp:
    for x, y in zip(X_train, y_train):
        fp.write('{} {}\n'.format(y, x))

with open('test.txt', 'w') as fp:
    for x, y in zip(X_test, y_test):
        fp.write('{} {}\n'.format(y, x))

# encode label
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print(list(label_encoder.classes_), '\n')
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

print(X_train[0], y_train[0], '\n')
print(X_test[0], y_test[0])

['0', '1'] 

Mình mua con này được vài hôm, đang dùng thử, có vài nhận xét thế này: Con này dùng ổn, pin trâu bò, nghe nhạc ngon, cảm ứng điện dung đa điểm hay vãi, giao diện nhắn tin có chức năng xem các tin nhắn gửi đi - nhận về kiểu "chat", nhìn hay hay.. màn hình thì khi xem video, xem ảnh, nhắn tin, thì có thể xoay ngang được. Wifi thì hơi khó kết nối, nhưng đã kết nối được rồi thì chạy ầm ầm, lướt web ngon lành, xem youtube qua wifi thì không hề giật tí nào... bluetooth gửi và nhận thì nhanh 1 

nexus chất quá 1


In [ ]:
MODEL_PATH = "models"

import os
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

In [ ]:
import pickle
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Naive Bayes in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "naive_bayes.pkl"), 'wb'))

Done training Naive Bayes in 0.42997169494628906 seconds.


In [ ]:
from sklearn.linear_model import LogisticRegression
    
start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs', 
                                                multi_class='auto',
                                                max_iter=10000))
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Linear Classifier in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "linear_classifier.pkl"), 'wb'))

Done training Linear Classifier in 0.2391195297241211 seconds.


In [ ]:
from sklearn.svm import SVC

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(gamma='scale'))
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training SVM in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join("/content/models/", "svm.pkl"), 'wb'))

Done training SVM in 2.247464418411255 seconds.


In [ ]:
import numpy as np
# Naive Bayes
model = pickle.load(open(os.path.join("/content/models/naive_bayes.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('Naive Bayes, Accuracy =', np.mean(y_pred == y_test))

# Linear Classifier
model = pickle.load(open(os.path.join("/content/models/linear_classifier.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('Linear Classifier, Accuracy =', np.mean(y_pred == y_test))

# SVM
model = pickle.load(open(os.path.join("/content/models/svm.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('SVM, Accuracy =', np.mean(y_pred == y_test))

Naive Bayes, Accuracy = 0.7493765586034913
Linear Classifier, Accuracy = 0.830423940149626
SVM, Accuracy = 0.8628428927680798


In [ ]:
# Xem kết quả trên từng nhãn
from sklearn.metrics import classification_report

nb_model = pickle.load(open(os.path.join("/content/models/svm.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

              precision    recall  f1-score   support

           0       0.86      0.67      0.76       255
           1       0.86      0.95      0.90       547

    accuracy                           0.86       802
   macro avg       0.86      0.81      0.83       802
weighted avg       0.86      0.86      0.86       802



In [ ]:
# xem kết quả cho 1 văn bản model svm đã load ở trên

document = "Máy tính ngu"

document = tien_xu_li(document)
document = remove_stopwords(document)

label = nb_model
print('Predict label:', label_encoder.inverse_transform(label))

Predict label: ['0']
